In [1]:
!pip install mne
!pip install mne-icalabel
!pip install keras-tuner



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Import necessary libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mne
from mne.preprocessing import ICA
from mne_icalabel import label_components
import re
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from keras_tuner.tuners import RandomSearch
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

# Constants
FS = 128  # Sampling frequency
METADATA_COLUMNS =  [
    'EEG.Counter', 'EEG.Interpolated', 'EEG.RawCq', 'EEG.Battery',
    'EEG.BatteryPercent', 'EEG.MarkerHardware'
]

print("Libraries imported and constants initialized.")

Libraries imported and constants initialized.


In [ ]:
import os
import pandas as pd
import re

# Function to load a single participant CSV file
def load_single_csv(file_path):
    if not isinstance(file_path, str) or not os.path.isfile(file_path) or not file_path.endswith('.csv'):
        print(f"❌ Invalid file: {file_path} (Must be a valid .csv file)")
        return None

    print(f"📂 Reading file: {file_path}")
    try:
        # Read CSV while skipping the first row
        df = pd.read_csv(file_path, skiprows=1)

        # Extract participant name from the filename using regex
        filename = os.path.basename(file_path)
        match = re.search(r'(par\.\d+)', filename)
        participant_name = match.group(0) if match else "Unknown"

        print(f"🆔 Assigning participant name: {participant_name}")
        df['Participant'] = participant_name  # Add participant name column

        return df

    except Exception as e:
        print(f"⚠️ Error reading {file_path}: {e}")
        return None
participant_data = load_single_csv(r"C:/Users/malak/OneDrive/Desktop/letters.csv")
# Use the injected CSV_FILE_PATH from Flask API
# if 'CSV_FILE_PATH' in globals():
#     global participant_data
#     participant_data = load_single_csv(CSV_FILE_PATH)
#     # single_csv_data = load_single_csv(CSV_FILE_PATH)
    
#     # Print a preview of the loaded data
#     if participant_data is not None:
#         print("✅ Data Loaded Successfully:")
#         print(participant_data.head())
#     else:
#         print("❌ Failed to load CSV data.")
# else:
#     print("⚠️ No CSV file path provided. Ensure CSV_FILE_PATH is set.")


📂 Reading file: C:/Users/User/Desktop/letters.csv
🆔 Assigning participant name: Unknown


In [5]:
# Function to validate EEG data for a single DataFrame
def validate_eeg_data(df):
    if df is None or df.empty:
        print("Received an empty DataFrame. Skipping validation.")
        return None, None

    # Identify EEG signal columns (those starting with 'EEG.' and not in metadata)
    valid_columns = [col for col in df.columns if col.startswith('EEG.') and col not in METADATA_COLUMNS]

    if not valid_columns:
        print(f"No valid EEG signal columns identified. Skipping this DataFrame.")
        return None, None  # Return None for both validated data and valid columns

    print(f"Validating data with shape: {df.shape}")
    return df[valid_columns].copy(), valid_columns  # Return DataFrame with valid EEG columns

# Validate the EEG data for the single participant
if participant_data is not None:
    validated_data, valid_columns = validate_eeg_data(participant_data)

    if validated_data is not None:
        print(f"Validated EEG Data:\n{validated_data.head()}")  # Display a preview
    else:
        print("No valid EEG data found.")
else:
    print("Failed to load CSV file.")


Validating data with shape: (1384, 69)
Validated EEG Data:
       EEG.AF3       EEG.F7       EEG.F3      EEG.FC5       EEG.T7  \
0  4259.358887  4317.563965  4332.179688  4329.230957  4306.025879   
1  4264.102539  4319.358887  4336.538574  4336.538574  4305.128418   
2  4282.179688  4332.307617  4347.820313  4343.717773  4310.512695   
3  4299.743652  4339.487305  4362.307617  4348.974121  4315.256348   
4  4306.538574  4338.077148  4369.871582  4354.102539  4311.922852   

        EEG.P7       EEG.O1       EEG.O2       EEG.P8       EEG.T8  \
0  4296.410156  4291.922852  4289.615234  4285.769043  4236.153809   
1  4304.358887  4295.769043  4289.615234  4290.512695  4243.717773   
2  4314.358887  4295.769043  4293.974121  4295.000000  4249.743652   
3  4311.666504  4297.948730  4294.615234  4295.384766  4251.282227   
4  4306.666504  4299.358887  4292.820313  4287.563965  4256.410156   

       EEG.FC6       EEG.F4       EEG.F8      EEG.AF4  
0  4343.589844  4427.436035  4305.769043  4

In [6]:
# Define channels to always drop
CHANNELS_TO_ALWAYS_DROP = ['EEG.O2', 'EEG.FC6']

# Function to detect bad channels
def detect_bad_channels(eeg_data, valid_columns, z_threshold=3.0):
    if eeg_data is None or eeg_data.empty:
        print("No EEG data available for bad channel detection.")
        return []

    bad_channels = []
    channel_variances = eeg_data.var(axis=0)
    z_scores = (channel_variances - channel_variances.mean()) / channel_variances.std()

    for idx, z in enumerate(z_scores):
        if abs(z) > z_threshold:
            bad_channels.append(valid_columns[idx])

    print(f"Detected bad channels: {bad_channels}")
    return bad_channels

# Function to clean specified bad channels and always drop O2 and FC6
def clean_bad_channels(eeg_data, bad_channels):
    if eeg_data is None or eeg_data.empty:
        print("No EEG data available for cleaning.")
        return None

    all_bad_channels = set(bad_channels + CHANNELS_TO_ALWAYS_DROP)  # Combine both lists
    eeg_data_cleaned = eeg_data.drop(columns=[col for col in all_bad_channels if col in eeg_data.columns])

    print(f"Dropped bad channels: {all_bad_channels}")
    print(f"Shape after cleaning: {eeg_data_cleaned.shape}")
    return eeg_data_cleaned

# Process single participant's EEG data
def process_single_participant(validated_data, valid_columns, z_threshold=3.0):
    if validated_data is None or validated_data.empty:
        print("No valid EEG data available for processing.")
        return None, []

    print("\nDetecting bad channels...")
    bad_channels = detect_bad_channels(validated_data, valid_columns, z_threshold)

    print("\nCleaning bad channels (including O2 and FC6)...")
    cleaned_data = clean_bad_channels(validated_data, bad_channels)

    print("\nFinished processing participant's EEG data.")
    return cleaned_data, bad_channels

# Example usage
if validated_data is not None:
    cleaned_data, bad_channels = process_single_participant(validated_data, valid_columns)
    print("Final cleaned EEG data and bad channels are ready for use.")



Detecting bad channels...
Detected bad channels: []

Cleaning bad channels (including O2 and FC6)...
Dropped bad channels: {'EEG.FC6', 'EEG.O2'}
Shape after cleaning: (1384, 12)

Finished processing participant's EEG data.
Final cleaned EEG data and bad channels are ready for use.


In [8]:
import mne

# Function to create an MNE Raw object
def create_raw_object(eeg_data):
    if eeg_data is None or eeg_data.empty:
        print("No EEG data available to create an MNE Raw object.")
        return None

    data = eeg_data.to_numpy().T
    ch_names = eeg_data.columns.tolist()
    info = mne.create_info(ch_names=ch_names, sfreq=FS, ch_types=['eeg'] * len(ch_names))
    raw = mne.io.RawArray(data, info)
    print("Created MNE Raw object.")
    return raw

# Function to filter rename_dict to exclude bad channels
def filter_rename_dict(rename_dict, bad_channels):
    return {key: value for key, value in rename_dict.items() if key not in bad_channels}

# Process cleaned data to create an MNE Raw object for the single participant
if cleaned_data is not None:
    print("\nCreating MNE Raw object for the participant...")

    try:
        raw_object = create_raw_object(cleaned_data)

        # Define standard channel renaming dictionary
        rename_dict = {
            'EEG.AF3': 'AF3', 'EEG.AF4': 'AF4', 'EEG.F7': 'F7',
            'EEG.F3': 'F3', 'EEG.FC5': 'FC5', 'EEG.T7': 'T7',
            'EEG.P7': 'P7', 'EEG.O1': 'O1',
            'EEG.P8': 'P8', 'EEG.T8': 'T8',
            'EEG.F4': 'F4', 'EEG.F8': 'F8'
        }

        # Filter out bad channels from rename_dict
        filtered_rename_dict = filter_rename_dict(rename_dict, bad_channels)

        if raw_object:
            raw_object.rename_channels(filtered_rename_dict)
            print("Successfully created and renamed MNE Raw object for the participant.")

    except Exception as e:
        print(f"Failed to create Raw object: {e}")
else:
    print("No cleaned EEG data available for MNE processing.")



Creating MNE Raw object for the participant...
Creating RawArray with float64 data, n_channels=12, n_times=1384
    Range : 0 ... 1383 =      0.000 ...    10.805 secs
Ready.
Created MNE Raw object.
Successfully created and renamed MNE Raw object for the participant.


In [9]:
import numpy as np
import mne
from mne.preprocessing import ICA
from mne_icalabel import label_components

# Apply filters and referencing to the Raw object
def apply_filters_and_reference(raw):
    raw.notch_filter(freqs=50.0)
    raw.filter(l_freq=1.0, h_freq=50.0)
    raw.set_eeg_reference('average', projection=True)

    # Remove non-EEG channels
    non_eeg_channels = ['EEG.Counter', 'EEG.Interpolated', 'EEG.Battery', 'EEG.BatteryPercent', 'EEG.MarkerHardware']
    existing_channels = [ch for ch in non_eeg_channels if ch in raw.ch_names]  # Only remove if they exist
    if existing_channels:
        raw.drop_channels(existing_channels)
        print(f"Removed non-EEG channels: {existing_channels}")

    # Apply standard 10-20 montage
    montage = mne.channels.make_standard_montage('standard_1020')
    raw.set_montage(montage, on_missing='ignore')

    print("Applied filters and average referencing.")
    return raw


# Perform ICA and clean artifacts
def perform_ica(raw, brain_threshold=0.5):
    if raw is None:
        print("No Raw object available for ICA.")
        return None

    ica = ICA(method='infomax', fit_params={'extended': True}, n_components=12, max_iter=1000)
    ica.fit(raw)
    ic_labels = label_components(raw, ica, method="iclabel")

    non_brain_components = [
        idx for idx, (label, prob) in enumerate(zip(ic_labels['labels'], ic_labels['y_pred_proba']))
        if label != 'brain' or prob < brain_threshold
    ]
    ica.exclude = non_brain_components
    raw_cleaned = ica.apply(raw)
    print(f"Excluded components: {non_brain_components}")
    return raw_cleaned

# Function to check and clean NaN/Inf values from raw data
def clean_nan_inf(raw):
    if raw is None:
        print("No Raw object available for NaN/Inf cleaning.")
        return None

    data = raw.get_data()
    if np.isnan(data).any() or np.isinf(data).any():
        print("Detected NaN or Inf values in the data.")
        for i in range(data.shape[0]):  # Iterate over channels
            channel_data = data[i, :]
            if np.isnan(channel_data).any() or np.isinf(channel_data).any():
                channel_mean = np.nanmean(channel_data)
                channel_data[np.isnan(channel_data)] = channel_mean
                channel_data[np.isinf(channel_data)] = channel_mean
                data[i, :] = channel_data
        print("Replaced NaN/Inf values with channel mean.")
    else:
        print("No NaN or Inf values detected.")

    raw._data = data  # Update the raw object
    return raw

# Process the single participant's Raw object
if raw_object is not None:
    print("\nProcessing Raw object for the participant...")

    try:
        # Apply filters and reference
        filtered_and_referenced_raw = apply_filters_and_reference(raw_object)

        # Clean NaN/Inf values
        cleaned_raw = clean_nan_inf(filtered_and_referenced_raw)

        # Perform ICA and clean artifacts
        cleaned_raw = perform_ica(cleaned_raw)

        print("Successfully processed and cleaned the Raw object for the participant.")

    except Exception as e:
        print(f"Failed to process Raw object: {e}")

else:
    print("No valid Raw object available for processing.")



Processing Raw object for the participant...
Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 845 samples (6.602 s)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cut

C:\Users\User\AppData\Local\Temp\ipykernel_9948\2678825756.py:35: RuntimeWarning: The provided Raw instance does not seem to be referenced to a common average reference (CAR). ICLabel was designed to classify features extracted from an EEG dataset referenced to a CAR (see the 'set_eeg_reference()' method for Raw and Epochs instances).
  ic_labels = label_components(raw, ica, method="iclabel")
C:\Users\User\AppData\Local\Temp\ipykernel_9948\2678825756.py:35: RuntimeWarning: The provided Raw instance is not filtered between 1 and 100 Hz. ICLabel was designed to classify features extracted from an EEG dataset bandpass filtered between 1 and 100 Hz (see the 'filter()' method for Raw and Epochs instances).
  ic_labels = label_components(raw, ica, method="iclabel")


Applying ICA to Raw instance
    Transforming to ICA space (12 components)
    Zeroing out 5 ICA components
    Projecting back using 12 PCA components
Excluded components: [0, 4, 7, 8, 10]
Successfully processed and cleaned the Raw object for the participant.


In [ ]:
import pandas as pd
import mne

# Interpolation and NaN handling for the Raw object after ICA and filtering
def interpolate_and_handle_nans(raw_cleaned):
    try:
        if raw_cleaned is None:
            print("No Raw object available for interpolation and NaN handling.")
            return None

        # Step 1: Interpolate bad channels
        bad_channels = raw_cleaned.info['bads']
        raw_cleaned.interpolate_bads(reset_bads=True)  # Interpolate bad channels
        # raw_cleaned.plot(n_channels=12, scalings=dict(eeg=50e-6))

        # Step 2: Handle NaN values in the EEG data
        eeg_data = pd.DataFrame(raw_cleaned.get_data().T, columns=raw_cleaned.ch_names)
        nan_locations = eeg_data.isna()

        # Print locations of NaN values
        if nan_locations.any().any():
            print("NaN values found in the following locations:")
            print(nan_locations)

        # Step 3: Interpolate NaN values in the EEG data
        eeg_data_interpolated = eeg_data.interpolate(method='linear', axis=0)

        # Step 4: Check if any NaN values remain after interpolation
        nan_check = eeg_data_interpolated.isna().sum()
        print("Remaining NaN values after interpolation per column:")
        print(nan_check)

        # Step 5: Replace remaining NaNs with forward fill or backward fill
        eeg_data_interpolated = eeg_data_interpolated.fillna(method='ffill').fillna(method='bfill')

        # Final check for remaining NaN values
        remaining_nans = eeg_data_interpolated.isna().sum()
        print(f"Remaining NaN values after interpolation: {remaining_nans}")

        # Step 6: Mark and interpolate bad channels in the Raw object
        raw_cleaned.info['bads'] = [ch for ch in bad_channels if ch in raw_cleaned.info['ch_names']]
        raw_cleaned.interpolate_bads(reset_bads=True)

        # Update Raw object with interpolated EEG data
        raw_cleaned = mne.io.RawArray(eeg_data_interpolated.T.to_numpy(), raw_cleaned.info)
        return raw_cleaned

    except Exception as e:
        print(f"Error during interpolation and NaN handling: {e}")
        return raw_cleaned

# Process the single participant's Raw object
if cleaned_raw is not None:
    print("\nProcessing Raw object for the participant...")

    try:
        # Apply interpolation and NaN handling
        interpolated_raw = interpolate_and_handle_nans(cleaned_raw)

        print("Successfully processed and cleaned the Raw object after interpolation and NaN handling.")

    except Exception as e:
        print(f"Failed to process Raw object: {e}")

else:
    print("No valid Raw object available for processing.")



Processing Raw object for the participant...
Setting channel interpolation method to {'eeg': 'spline'}.


Remaining NaN values after interpolation per column:
AF3    0
F7     0
F3     0
FC5    0
T7     0
P7     0
O1     0
P8     0
T8     0
F4     0
F8     0
AF4    0
dtype: int64
Remaining NaN values after interpolation: AF3    0
F7     0
F3     0
FC5    0
T7     0
P7     0
O1     0
P8     0
T8     0
F4     0
F8     0
AF4    0
dtype: int64
Setting channel interpolation method to {'eeg': 'spline'}.
Creating RawArray with float64 data, n_channels=12, n_times=1384
    Range : 0 ... 1383 =      0.000 ...    10.805 secs
Ready.
Successfully processed and cleaned the Raw object after interpolation and NaN handling.


C:\Users\User\AppData\Local\Temp\ipykernel_9948\845367928.py:13: RuntimeWarning: No bad channels to interpolate. Doing nothing...
  raw_cleaned.interpolate_bads(reset_bads=True)  # Interpolate bad channels
C:\Users\User\AppData\Local\Temp\ipykernel_9948\845367928.py:34: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  eeg_data_interpolated = eeg_data_interpolated.fillna(method='ffill').fillna(method='bfill')
C:\Users\User\AppData\Local\Temp\ipykernel_9948\845367928.py:42: RuntimeWarning: No bad channels to interpolate. Doing nothing...
  raw_cleaned.interpolate_bads(reset_bads=True)


In [11]:
import numpy as np

def z_score_normalize_eeg_data(raw):
    """
    Normalize the EEG data in the Raw object using Z-score normalization
    (zero mean, unit variance), avoiding division by zero.
    """
    eeg_data = raw.get_data()  # Extract EEG data
    mean = np.mean(eeg_data, axis=1, keepdims=True)  # Compute the mean for each channel

    # Compute standard deviation, adding a small epsilon to avoid division by zero
    epsilon = 1e-8  # Small value to prevent division by zero
    std = np.std(eeg_data, axis=1, keepdims=True) + epsilon

    # Z-score normalization
    normalized_data = (eeg_data - mean) / std

    # Replace the data in the Raw object with normalized data
    raw._data = normalized_data
    print("Applied Z-score normalization (zero mean, unit variance).")
    return raw

# Apply Z-score normalization to one participant's EEG data
participant_raw = interpolated_raw  # Make sure this contains the participant's Raw object

try:
    z_score_normalized_raw_participant = z_score_normalize_eeg_data(participant_raw)
    print("Successfully applied Z-score normalization to the participant's EEG data.")

    # Check statistics to confirm normalization
    data = z_score_normalized_raw_participant.get_data()
    print(f"Mean of Z-score normalized data: {np.mean(data, axis=1)}")
    print(f"Standard deviation of Z-score normalized data: {np.std(data, axis=1)}")

except Exception as e:
    print(f"Failed to apply Z-score normalization: {e}")


Applied Z-score normalization (zero mean, unit variance).
Successfully applied Z-score normalization to the participant's EEG data.
Mean of Z-score normalized data: [ 5.13397930e-18  2.56698965e-17 -1.54019379e-17  1.54019379e-17
 -1.02679586e-17  1.54019379e-17  2.56698965e-18  1.79689276e-17
 -3.59378551e-17 -2.82368862e-17 -5.13397930e-18 -2.05359172e-17]
Standard deviation of Z-score normalized data: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [ ]:
import os
import pandas as pd
import numpy as np
import mne

# Segment length in seconds
SEGMENT_LENGTH_SECONDS = 2
SEGMENT_OVERLAP = 0.5  # Half-second overlap between segments
segment_length = FS * SEGMENT_LENGTH_SECONDS
overlap_length = int(FS * SEGMENT_OVERLAP)  # Overlap in samples

segmented_participant_data = []

print("Segmenting EEG data for the participant...")

if isinstance(z_score_normalized_raw_participant, mne.io.RawArray):
    data = z_score_normalized_raw_participant.get_data().T
    num_segments = (len(data) - overlap_length) // (segment_length - overlap_length)

    # Create segments with overlap
    for i in range(num_segments):
        start = i * (segment_length - overlap_length)
        end = start + segment_length
        segments = data[start:end]
        segmented_participant_data.append(segments)

    # Handle remaining partial segment if it exists
    if len(data) % (segment_length - overlap_length) != 0:
        partial_segment = data[-segment_length:]  # Take the last full segment or as much as we can
        print(f"Partial segment with length {len(partial_segment)}")
        segmented_participant_data.append(partial_segment)

    print(f"{len(segmented_participant_data)} segments created.")

else:
    print(f"Unsupported data type: {type(z_score_normalized_raw_participant)}")

# Convert segments to DataFrame
flat_segments = [pd.DataFrame(seg, columns=[f"Channel_{i}" for i in range(seg.shape[1])]) for seg in segmented_participant_data]

if flat_segments:
    df = pd.concat(flat_segments, ignore_index=True)

# Save to CSV
# Define a path to save the segmented data
SEGMENTED_CSV_PATH = os.path.join(os.getcwd(), "segmented_eeg_data.csv")
df.to_csv(SEGMENTED_CSV_PATH, index=False)
print(f"Segmented data saved as {SEGMENTED_CSV_PATH}")


Segmenting EEG data for the participant...
Partial segment with length 104
5 full segments created.
Segmented data saved as c:\Users\User\Neurithm\notebooks\segmented_eeg_data.csv
